In [ ]:
## module python permettant d'utiliser le protocole http
!pip install requests
## package is used to automate web browser interaction from Python
!pip install selenium==3.5.0
## library that makes it easy to scrape information from web pages
!pip install beautifulsoup4

In [8]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## to lunch the web chrome driver
driver = webdriver.Chrome(
    executable_path="./chromedriver.exe"
    )

In [36]:
from bs4 import BeautifulSoup
import re

## a list that conatins all the small case letters
atoz = []
for i in range(97,123):
    atoz.append(chr(i))

## all players url
url_All_Players = "https://www.basketball-reference.com/players/{}/"

## all players urls list
All_players_urls_list = []

## extract all the html pages that contains players urls
"""for letter in atoz:
    url = url_All_Players.format(letter)

    ## enter url
    driver.get(url)
    ## scroll down the page
    driver.execute_script("window.scrollTo(1,10000)")
    ## 1 sec pause (the website does not allow fast scrapping and detects automats)
    time.sleep(1)

    ## save html pages
    with open("atoz_pages/{}.html".format(letter), "w+", encoding="utf-8") as f:
        f.write(driver.page_source)"""


## extract each players url (we need the url to get his stats later)
## go through the html pages from a to z 
for letter in atoz:
    with open("atoz_pages/{}.html".format(letter), encoding="utf-8") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    
    all_players = soup.find_all(id="players")[0]
    ## extract the players link
    for link, year_max, year_min in zip(all_players.find_all('a', href=re.compile(r'/players/[a-z]/[a-zA-Z0-9_]+\.html')), all_players.find_all("td", attrs={"data-stat": "year_max"}), all_players.find_all("td", attrs={"data-stat": "year_min"})):
        
        ## get the year when the player stopped playing
        y_max = year_max.text.strip()
        y_min = year_min.text.strip()
        ## test the year when the player stopped playing
        ## we remove all the players that didn't play from 1993 to 2024
        if int(y_max) >= 1993 and int(y_min) < 2024 :
            href = link['href']
            player_url = "https://www.basketball-reference.com" + href
            All_players_urls_list.append(player_url)

In [37]:
All_players_urls_list

['https://www.basketball-reference.com/players/a/abdelal01.html',
 'https://www.basketball-reference.com/players/a/abdulma02.html',
 'https://www.basketball-reference.com/players/a/abdulta01.html',
 'https://www.basketball-reference.com/players/a/abdursh01.html',
 'https://www.basketball-reference.com/players/a/abrinal01.html',
 'https://www.basketball-reference.com/players/a/achiupr01.html',
 'https://www.basketball-reference.com/players/a/ackeral01.html',
 'https://www.basketball-reference.com/players/a/acresma01.html',
 'https://www.basketball-reference.com/players/a/acyqu01.html',
 'https://www.basketball-reference.com/players/a/adamsha01.html',
 'https://www.basketball-reference.com/players/a/adamsja01.html',
 'https://www.basketball-reference.com/players/a/adamsjo01.html',
 'https://www.basketball-reference.com/players/a/adamsmi01.html',
 'https://www.basketball-reference.com/players/a/adamsst01.html',
 'https://www.basketball-reference.com/players/a/addisra01.html',
 'https://ww

In [38]:
len(All_players_urls_list)

2789

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## to lunch the web chrome driver
driver = webdriver.Chrome(
    executable_path="./chromedriver.exe"
    )

In [9]:
listp = ['https://www.basketball-reference.com/players/j/jacksja02.html']

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re

## importing additional datasets
raptor_by_player_stats = pd.read_csv("Addition/RAPTOR_by_player.csv")
espn_by_player_stats = pd.read_csv("Addition/espn_com_stats.csv")
nba_by_player_stats = pd.read_csv("Addition/nba_com_stats.csv")
bball_by_player_stats = pd.read_csv("Addition/bball_com_stats.csv")

## this loop will go over all the extracted data without additionals
for url in All_players_urls_list:
    ## get the player url
    player_url = url

    ## get players code from his url
    parts = url.split("/")
    player_code = parts[-1].replace(".html", "")

    ## load players stats without additional stats
    player_stat_wo_additional = pd.read_csv("All_players_stats/{0}/player_team_stats.csv".format(player_code))
    
    ## getting the current player from raptor stats
    raptor_player = raptor_by_player_stats[raptor_by_player_stats['player_id'] == player_code ]
    raptor_player = raptor_player.add_suffix('_raptor')
    raptor_player.reset_index(drop=True, inplace=True)
    
    
    ## getting the current player from espn stats
    espn_player = espn_by_player_stats[espn_by_player_stats['NAME'].str.split(',').str[0] == player_stat_wo_additional['Player_name'].iloc[0]]
    espn_player = espn_player.add_suffix('_espn')
    espn_player.reset_index(drop=True, inplace=True)
    
    ## getting the current player from nba stats
    nba_player = nba_by_player_stats[nba_by_player_stats['PLAYER'] == player_stat_wo_additional['Player_name'].iloc[0]]
    nba_player = nba_player.add_suffix('_nba')
    nba_player.reset_index(drop=True, inplace=True)
    
    ## getting the current player from bball stats
    bball_player = bball_by_player_stats[bball_by_player_stats['Player'] == player_stat_wo_additional['Player_name'].iloc[0].rstrip('.')]
    bball_player = bball_player.sort_values(by='Season', ascending=True)
    bball_player = bball_player.add_suffix('_bball')
    bball_player.reset_index(drop=True, inplace=True)

    columns_to_exclude_raptor = ['player_name_raptor', 'player_id_raptor', 'season_raptor'] 
    columns_to_exclude_espn = ['RK_espn', 'NAME_espn', 'TEAM_espn', 'Year_espn']
    columns_to_exclude_nba = ['PLAYER_nba', 'TEAM_nba', 'AGE_nba', 'Year_nba']
    column_to_keep = 'D-LEBRON_bball' 

    ## creating empty dataaframes
    raptor_player_stat = pd.DataFrame()
    espn_player_stat = pd.DataFrame()
    nba_player_stat = pd.DataFrame()
    bball_player_stat = pd.DataFrame()
    
    for row in player_stat_wo_additional.itertuples(index=False):
        year = int(row.Year)
        team = str(row.Tm)
        season = row.Season
        
        # RAPTOR
        # Filter rows in raptor_player based on the 'season_raptor' column
        raptor_row = raptor_player[raptor_player['season_raptor'] == year]
        if raptor_row.empty:
            raptor_row.loc[0] = 0
        raptor_row = raptor_row.drop(columns=columns_to_exclude_raptor)
        
        raptor_player_stat = pd.concat([raptor_player_stat,raptor_row], axis=0)

        # ESPN
        # Filter rows in espn_player based on the 'Year_espn' column
        espn_row = espn_player[espn_player['Year_espn'] == year]
        if espn_row.empty:
            espn_row.loc[0] = 0 
        espn_row = espn_row.drop(columns=columns_to_exclude_espn)
        espn_row = espn_row.drop(columns=espn_row.columns[espn_row.columns.str.contains('Unnamed:')])
        espn_row = espn_row.drop_duplicates()
        espn_player_stat = pd.concat([espn_player_stat,espn_row], axis=0)

        # NBA
        # Filter rows in nba_player based on the 'Year_nba' column
        nba_row = nba_player[(nba_player['Year_nba'] == year)]
        
        if nba_row.empty:
            nba_row.loc[0] = 0    
        nba_row = nba_row.drop(columns=columns_to_exclude_nba)
        
        nba_player_stat = pd.concat([nba_player_stat,nba_row], axis=0)
        
        # BBALL INDEX
        # Filter rows in bball_player based on the 'Season_bball' column
        bball_row = bball_player[(bball_player['Season_bball'] == season)]
        
        if bball_row.empty:
            bball_row.loc[0] = 0    
        # Drop all columns except the one you want to keep
        bball_row.drop(columns=bball_row.columns.difference([column_to_keep]), inplace=True)

        bball_player_stat = pd.concat([bball_player_stat,bball_row], axis=0)

    raptor_player_stat.reset_index(drop=True, inplace=True)
    espn_player_stat.reset_index(drop=True, inplace=True)
    nba_player_stat.reset_index(drop=True, inplace=True)
    bball_player_stat.reset_index(drop=True, inplace=True)

    player_stat_w_additional = pd.concat([player_stat_wo_additional,espn_player_stat, raptor_player_stat, nba_player_stat,bball_player_stat], axis=1, ignore_index=False)
    player_stat_w_additional = player_stat_w_additional.drop(columns=player_stat_w_additional.columns[player_stat_w_additional.columns.str.contains('Unnamed:')])
    player_stat_w_additional.reset_index(drop=True, inplace=True)
    
    player_stat_w_additional.to_csv("All_players_stats_with_additional/{}.csv".format(player_code))

In [40]:
## concat all the csv files of players into one big dataframe
all_players_w_add_df = pd.DataFrame()
for url in All_players_urls_list:
    parts = url.split("/")
    player_code = parts[-1].replace(".html", "")
    
    single_player_stats = pd.read_csv("All_players_stats_with_additional/{0}.csv".format(player_code))
    all_players_w_add_df = pd.concat([all_players_w_add_df, single_player_stats], axis=0, ignore_index=True)
    ##all_players_w_add_df.reset_index(drop=True, inplace=True)

all_players_w_add_df = all_players_w_add_df.drop(columns=all_players_w_add_df.columns[all_players_w_add_df.columns.str.contains('Unnamed:')])

all_players_w_add_df.to_csv("all_players_with_additional.csv")

In [149]:
## concat all the csv files of players into one big dataframe
all_players_df = pd.DataFrame()
for url in All_players_urls_list:
    parts = url.split("/")
    player_code = parts[-1].replace(".html", "")
    
    player_stats = pd.read_csv("All_players_stats/{0}/player_team_stats.csv".format(player_code))
    all_players_df = pd.concat([all_players_df,player_stats], axis=0)
    all_players_df.reset_index(drop=True, inplace=True)
    
all_players_df.to_csv("all_players.csv")

In [150]:
all_players_df_sorted = all_players_df.sort_values(by=['Year'], ascending=[False])
all_players_df_sorted.reset_index(drop=True, inplace=True)
all_players_df_sorted.to_csv("all_players_sorted.csv")

In [45]:
all_players_w_add_df_sorted = all_players_w_add_df.sort_values(by=['Year'], ascending=[False])
all_players_w_add_df_sorted.reset_index(drop=True, inplace=True)
all_players_w_add_df_sorted = all_players_w_add_df_sorted.iloc[:-11]
all_players_w_add_df_sorted.fillna(0)
display(all_players_w_add_df_sorted)
all_players_w_add_df_sorted.to_csv("all_players_w_add_sorted.csv")

,Season,Year,Player_name,Rank,Dpoy_titles,Dpoy_votes,First_votes,won_dpoy_last_season,teamates_with_dpoy,Points_won,...,STL_nba,STL%_nba,BLK_nba,%BLK_nba,OPP PTS OFF TOV_nba,OPP PTS 2ND CHANCE_nba,OPP PTS FB_nba,OPP PTS PAINT_nba,DEF WS_nba,D-LEBRON_bball
0,2022-23,2023.0,Ivica Zubac,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.4,9.8,1.3,50.5,10.6,7.4,9.0,29.4,0.080,2.07
1,2022-23,2023.0,Rodney McGruder,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.5,18.4,0.0,2.4,5.4,4.6,4.6,18.5,0.039,-0.49
2,2022-23,2023.0,DeMar DeRozan,-1.0,0.0,0.0,0.0,False,1.0,0.0,...,1.1,19.0,0.5,15.3,11.2,8.6,9.5,34.6,0.129,0.59
3,2022-23,2023.0,Vernon Carey Jr.,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.2,50.0,0.2,50.0,0.5,0.6,0.6,1.6,0.024,-0.06
4,2022-23,2023.0,D'Angelo Russell,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,1.0,19.9,0.4,11.3,12.3,9.6,9.1,34.3,0.102,-0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16279,1992-93,1993.0,Alec Kessler,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00
16280,1992-93,1993.0,Jerome Kersey,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00
16281,1992-93,1993.0,Kiki Vandeweghe,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00
16282,1992-93,1993.0,Steve Kerr,-1.0,0.0,0.0,0.0,False,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00
